In [1]:
#r "./bin/Debug/net7.0/studies.dll"
#r "./bin/Debug/net7.0/core.fs.dll"
#r "nuget:FSharp.Data"
#r "nuget: Plotly.NET.Interactive"

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed Packages FSharp.Data, 6.3.0 Plotly.NET.Interactive, 4.2.1

Loading extensions from `C:\Users\laimi\.nuget\packages\plotly.net.interactive\4.2.1\lib\netstandard2.1\Plotly.NET.Interactive.dll`

In [2]:
open FSharp.Data
open Plotly.NET

let trade_csv = "d:\\studies\\03_export_date_ticker_screenerid_gap_outcomes.csv"

let csv = studies.GapStudy.parseTradeOutcomes trade_csv

In [3]:
let outputSummary includeChart (summary:studies.Types.TradeSummary) = 

    printfn "%s: %i trades, %.2f%% win pct, %.2f%% avg gain, %.2f%% avg loss, %.2f avg gain/loss, %.2f EV" 
        summary.StrategyName summary.Total (summary.WinPct * 100m) (summary.AvgWin) (summary.AvgLoss) summary.AvgGainLoss summary.EV

    match includeChart with
    | true ->
        let chart = 
            Chart.Histogram(
                X = summary.Gains,
                NBinsX = 40,
                Name = "Gain Distribution"
            )

        chart.Display() |> ignore
    | false -> ()

In [4]:
let filterNamePairs = [
    ("All", fun (o:studies.Types.TradeOutcomeOutput.Row) -> true)
    ("New Highs", fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.Screenerid = 28)
    ("Top Gainers", fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.Screenerid = 29)
    ("Gap ups", fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.HasGapUp)
    ("Gap ups - new highs", fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.HasGapUp && o.Screenerid = 28)
    ("Gap ups - top gainers", fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.HasGapUp && o.Screenerid = 29)
]

let tradesGroupedByStrategy = csv |> Seq.groupBy (fun t -> t.Strategy) // all strategies, no filters

filterNamePairs
|> Seq.iter (
    fun (filterName,filter) ->

        printfn $"Trade: {filterName}"
        tradesGroupedByStrategy
        |> Seq.iter (
            fun (strategy, trades) ->
                let filteredTrades = trades |> Seq.filter filter
                let summary = studies.Types.TradeSummary.create filteredTrades
                outputSummary false summary
        )

        printfn ""
        printfn ""
)

Trade: All
Buy and Hold 5 bars: 22848 trades, 50.28% win pct, 7.40% avg gain, -6.87% avg loss, 1.08 avg gain/loss, 0.30 EV
Buy and Hold 10 bars: 22848 trades, 49.16% win pct, 9.15% avg gain, -9.07% avg loss, 1.01 avg gain/loss, -0.12 EV
Buy and Hold 30 bars: 22848 trades, 45.54% win pct, 14.08% avg gain, -14.67% avg loss, 0.96 avg gain/loss, -1.58 EV
Buy and Hold 60 bars: 22848 trades, 44.82% win pct, 18.24% avg gain, -18.96% avg loss, 0.96 avg gain/loss, -2.29 EV
Buy and Hold 90 bars: 22848 trades, 44.18% win pct, 20.46% avg gain, -21.37% avg loss, 0.96 avg gain/loss, -2.89 EV
Buy and Hold: 22848 trades, 43.08% win pct, 29.77% avg gain, -30.46% avg loss, 0.98 avg gain/loss, -4.52 EV


Trade: New Highs
Buy and Hold 5 bars: 8692 trades, 47.95% win pct, 4.57% avg gain, -4.50% avg loss, 1.02 avg gain/loss, -0.15 EV
Buy and Hold 10 bars: 8692 trades, 48.48% win pct, 5.86% avg gain, -6.27% avg loss, 0.94 avg gain/loss, -0.38 EV
Buy and Hold 30 bars: 8692 trades, 45.38% win pct, 9.53% avg ga

In [5]:
let data =
    filterNamePairs
    |> Seq.map (fun (filterName, filter) ->
        tradesGroupedByStrategy
        |> Seq.map (fun (strategy, trades) ->
            let filteredTrades = trades |> Seq.filter filter
            let summary = studies.Types.TradeSummary.create filteredTrades
            (strategy, filterName, summary)
        )
    )
    |> Seq.concat

let sortFunctions = [
    ("EV", fun (t:studies.Types.TradeSummary) -> t.EV)
    ("Win %", fun (t:studies.Types.TradeSummary) -> t.WinPct)
    ("Avg Gain", fun (t:studies.Types.TradeSummary) -> t.AvgGainLoss)
]

sortFunctions
|> Seq.iter (fun (sortLabel, sortFunction) ->
    printfn $"Top 5 by: {sortLabel}"
    data
    |> Seq.sortByDescending (fun (strategy, filterName, summary) -> sortFunction summary)
    |> Seq.take 5
    |> Seq.iter (fun (strategy, filterName, summary) ->
        printfn $"Strategy: {strategy}, Filter: {filterName}: {sortFunction summary}"
    )
    printfn ""
    printfn ""
)

Top 5 by: EV
Strategy: Buy and Hold, Filter: Gap ups - new highs: 1.822461023902737286555059309
Strategy: Buy and Hold, Filter: New Highs: 0.736784720611110804648903637
Strategy: Buy and Hold 5 bars, Filter: Top Gainers: 0.5825484022408140426570786786
Strategy: Buy and Hold 5 bars, Filter: Gap ups - top gainers: 0.5698105391586863628178409871
Strategy: Buy and Hold 5 bars, Filter: Gap ups: 0.4783539464164832646621860451


Top 5 by: Win %
Strategy: Buy and Hold 5 bars, Filter: Top Gainers: 0.5171658660638598474145238768
Strategy: Buy and Hold 5 bars, Filter: Gap ups - top gainers: 0.5154989384288747346072186837
Strategy: Buy and Hold 10 bars, Filter: Gap ups - top gainers: 0.5131634819532908704883227176
Strategy: Buy and Hold 10 bars, Filter: Gap ups: 0.5081433224755700325732899023
Strategy: Buy and Hold 5 bars, Filter: Gap ups: 0.507057546145494028230184582


Top 5 by: Avg Gain
Strategy: Buy and Hold, Filter: Gap ups - new highs: 1.2635946275090736548676078349
Strategy: Buy and Hold 5 

In [8]:
// let's seelect random trades and double what's going on with them
let random = System.Random()

csv
|> Seq.sortByDescending (fun r -> r.PercentGain)
|> Seq.take 2
|> Seq.iter ( fun t -> 
    t.Display() |> ignore
)

(Buy and Hold, SMCI, 4/19/2022 12:00:00 AM, 29, True, 4/20/2022 12:00:00 AM, 44.79, 11/14/2023 12:00:00 AM, 282.25, 530.16298280866264791248046439, 573) Item1 Buy and Hold Item2 SMCI Item3 2022-04-19 00:00:00Z Item4 29 Item5 True Item6 2022-04-20 00:00:00Z Item7 44.79 Rest (11/14/2023 12:00:00 AM, 282.25, 530.16298280866264791248046439, 573) Item1 2023-11-14 00:00:00Z Item2 282.25 Item3 530.16298280866264791248046439 Item4 573

(Buy and Hold, EDU, 4/29/2022 12:00:00 AM, 29, True, 4/29/2022 12:00:00 AM, 11.6, 11/14/2023 12:00:00 AM, 69.42, 498.44827586206896551724137931, 564) Item1 Buy and Hold Item2 EDU Item3 2022-04-29 00:00:00Z Item4 29 Item5 True Item6 2022-04-29 00:00:00Z Item7 11.6 Rest (11/14/2023 12:00:00 AM, 69.42, 498.44827586206896551724137931, 564) Item1 2023-11-14 00:00:00Z Item2 69.42 Item3 498.44827586206896551724137931 Item4 564

In [7]:
csv |> Seq.map (fun (r:studies.Types.TradeOutcomeOutput.Row) -> r.Screenerid)

[ 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29 ... (more) ] f Microsoft.FSharp.Collections.SeqModule+revamp@728[System.Tuple`8[System.String,System.String,System.DateTime,System.Int32,System.Boolean,System.DateTime,System.Decimal,System.Tuple`4[System.DateTime,System.Decimal,System.Decimal,System.Int32]],System.Int32] f Microsoft.FSharp.Collections.SeqModule+Map@748-1[System.Tuple`8[System.String,System.String,System.DateTime,System.Int32,System.Boolean,System.DateTime,System.Decimal,System.Tuple`4[System.DateTime,System.Decimal,System.Decimal,System.Int32]],System.Int32] mapping FSI_0010+it@1-5 ie [ (Buy and Hold 5 bars, ACET, 3/22/2022 12:00:00 AM, 29, False, 3/23/2022 12:00:00 AM, 16.73, 3/30/2022 12:00:00 AM, 19.57, 16.975493126120741183502689780, 7), (Buy and Hold 10 bars, ACET, 3/22/2022 12:00:00 AM, 29, False, 3/23/2022 12:00:00 AM, 16.73, 4/6/2022 12:00:00 AM, 19.38, 15.839808726838015540944411240, 14), (Buy and Hold 30 bars, ACET, 3/22/2022 12:00:00 AM, 29, False, 3/23/2022 12:00:00 AM, 16.73, 5/5/2022 12:00:00 AM, 15.34, -8.308427973699940227136879860, 43), (Buy and Hold 60 bars, ACET, 3/22/2022 12:00:00 AM, 29, False, 3/23/2022 12:00:00 AM, 16.73, 6/17/2022 12:00:00 AM, 11.72, -29.946204423191870890615660490, 86), (Buy and Hold 90 bars, ACET, 3/22/2022 12:00:00 AM, 29, False, 3/23/2022 12:00:00 AM, 16.73, 8/2/2022 12:00:00 AM, 17.21, 2.8690974297668858338314405300, 132), (Buy and Hold, ACET, 3/22/2022 12:00:00 AM, 29, False, 3/23/2022 12:00:00 AM, 16.73, 11/14/2023 12:00:00 AM, 1.17, -93.00657501494321578003586372, 601), (Buy and Hold 5 bars, ACMR, 3/22/2022 12:00:00 AM, 29, False, 3/23/2022 12:00:00 AM, 21.29666666666667, 3/30/2022 12:00:00 AM, 21.47, 0.8138988887149630906403367200, 7), (Buy and Hold 10 bars, ACMR, 3/22/2022 12:00:00 AM, 29, False, 3/23/2022 12:00:00 AM, 21.29666666666667, 4/6/2022 12:00:00 AM, 20.25, -4.914697135702002674174810500, 14), (Buy and Hold 30 bars, ACMR, 3/22/2022 12:00:00 AM, 29, False, 3/23/2022 12:00:00 AM, 21.29666666666667, 5/5/2022 12:00:00 AM, 16.03, -24.730004695570523598371467280, 43), (Buy and Hold 60 bars, ACMR, 3/22/2022 12:00:00 AM, 29, False, 3/23/2022 12:00:00 AM, 21.29666666666667, 6/17/2022 12:00:00 AM, 13.92, -34.637658475504784060469795660, 86), (Buy and Hold 90 bars, ACMR, 3/22/2022 12:00:00 AM, 29, False, 3/23/2022 12:00:00 AM, 21.29666666666667, 8/2/2022 12:00:00 AM, 16.78, -21.208326811707634808526089890, 132), (Buy and Hold, ACMR, 3/22/2022 12:00:00 AM, 29, False, 3/23/2022 12:00:00 AM, 21.29666666666667, 11/14/2023 12:00:00 AM, 17.7198, -16.795429644701844295597175660, 601), (Buy and Hold 5 bars, AMC, 3/22/2022 12:00:00 AM, 29, False, 3/23/2022 12:00:00 AM, 187.5, 3/30/2022 12:00:00 AM, 256.8, 36.9600, 7), (Buy and Hold 10 bars, AMC, 3/22/2022 12:00:00 AM, 29, False, 3/23/2022 12:00:00 AM, 187.5, 4/6/2022 12:00:00 AM, 203.9, 8.746666666666666666666666670, 14), (Buy and Hold 30 bars, AMC, 3/22/2022 12:00:00 AM, 29, False, 3/23/2022 12:00:00 AM, 187.5, 5/5/2022 12:00:00 AM, 146.9, -21.653333333333333333333333330, 43), (Buy and Hold 60 bars, AMC, 3/22/2022 12:00:00 AM, 29, False, 3/23/2022 12:00:00 AM, 187.5, 6/17/2022 12:00:00 AM, 125.3, -33.173333333333333333333333330, 86), (Buy and Hold 90 bars, AMC, 3/22/2022 12:00:00 AM, 29, False, 3/23/2022 12:00:00 AM, 187.5, 8/2/2022 12:00:00 AM, 168.6, -10.0800, 132), (Buy and Hold, AMC, 3/22/2022 12:00:00 AM, 29, False, 3/23/2022 12:00:00 AM, 187.5, 11/14/2023 12:00:00 AM, 8.1701, -95.64261333333333333333333333, 601), (Buy and Hold 5 bars, AOSL, 3/22/2022 12:00:00 AM, 29, False, 3/23/2022 12:00:00 AM, 62.57, 3/30/2022 12:00:00 AM, 63.02, 0.7191945021575835064727505200, 7), (Buy and Hold 10 bars, AOSL, 3/22/2022 12:00:00 AM, 29, False, 3/23/2022 12:00:00 AM, 62.57, 4/6/2022 12:00:00 AM, 46.44, -25.779127377337382132012146400, 14) ... (more) ] (values) index value 0 (Buy and Hold 5 bars, ACET, 3/22/2022 12:00:00 AM, 29, False, 3/23/2022 12:00:00 AM, 16.73, 3/30/2022 12:00:00 AM, 19.57, 16.9